In [15]:
import pandas as pd
import pyodbc
username = 'vigrose'
password = 'Hufflepuff10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [7]:
load = pd.read_csv('../../Data/AMAIA/AMAIA_STG_AMA_ADDR.txt', encoding='latin-1')
og = pd.read_excel('../../Data/AMAIA/AMAIA_LOAD_2020-09-10.xlsx')

In [16]:
sql_query = \
    '''
    SELECT DISTINCT 
    M.ME,
    E.ENTITY_ID
    FROM 
    AMAEDW.PARTY_ID_TO_ME_VW M, AMAEDW.PARTY_ENTITY_VW E
    WHERE 
    M.PARTY_ID=E.PARTY_ID
    '''
me_entity = pd.read_sql(con=AMAEDW, sql=sql_query)

In [23]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums

In [24]:
og['ME']=fix_me(og['me#'])

In [26]:
og = pd.merge(og, me_entity, on='ME')

In [8]:
len(load)

94558

In [9]:
len(og)

94877

In [31]:
og =og.rename(columns={'ENTITY_ID':'entity_id'})

In [47]:
load = load.astype(str)

In [50]:
og = og.astype(str)

In [38]:
load['entity_id']=[str(x) for x in load.entity_id]

In [69]:
pd.merge(og, load, on = 'entity_id')[['comm_id_x', 'comm_id_y']]

,comm_id_x,comm_id_y
0,1448128,1448128
1,9461001,9461001
2,748979,24970372
3,6107649,6107649
4,6107649,6107649
...,...,...
94553,3141702,25062937
94554,2951631,25062938
94555,2862611,25062939
94556,2465546,25062940


In [62]:
load['addr_zip']=[x.replace('.0','') for x in load.addr_zip]

In [63]:
load[['entity_id','comm_id', 'load_type_ind', 'addr_type', 'addr_line_1',
       'addr_line_2', 'addr_line_3', 'addr_city', 'addr_state', 'addr_zip',
       'source']]

,entity_id,comm_id,load_type_ind,addr_type,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,source
0,5733607,1448128,R,N,nan,nan,158 Fox Meadow Rd,Scarsdale,NY,10583,INS-USG
1,12753048,9461001,R,N,nan,nan,1804 Leeds Dr,Southlake,TX,76092,INS-USG
2,43014578,24970372,R,N,1500 E Medical Center Dr,3119 Taubman Center,nan,Ann Arbor,MI,48109,INS-USG
3,46605698,6107649,R,N,nan,nan,4100 Lake Dr SE Ste 200,Grand Rapids,MI,49546,INS-USG
4,44903231,6107649,R,N,nan,nan,4100 Lake Dr SE Ste 200,Grand Rapids,MI,49546,INS-USG
...,...,...,...,...,...,...,...,...,...,...,...
94553,7488178,25062937,R,N,917 W Oak St,nan,nan,Greenville,MI,48838,INS-USG
94554,12827408,25062938,R,N,13668 Westheimer Rd,nan,nan,Houston,TX,77077,INS-USG
94555,12329105,25062939,R,N,1153 N Lavergne Ave,nan,nan,Chicago,IL,60651,INS-USG
94556,11136511,25062940,R,N,5041 Shirley Ave,nan,nan,Tarzana,CA,91356,INS-USG


In [55]:
og.head()

,me#,comm_id,usage,load_type_ind,addr_type,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,source,source_dtm,ME,entity_id
0,2512921232,1448128,PP,R,N,158 Fox Meadow Rd,nan,nan,Scarsdale,NY,10583,INS-USG,2020-09-10,02512921232,5733607
1,49514950015,9461001,PP,R,N,1804 Leeds Dr,nan,nan,Southlake,TX,76092,INS-USG,2020-09-10,49514950015,12753048
2,2802131341,748979,PP,R,N,1500 E Medical Center Dr,3119 Taubman Center,nan,Ann Arbor,MI,48109,INS-USG,2020-09-10,02802131341,43014578
3,2879111024,6107649,PP,R,N,4100 Lake Dr SE Ste 200,nan,nan,Grand Rapids,MI,49546,INS-USG,2020-09-10,02879111024,46605698
4,49537130317,6107649,PP,R,N,4100 Lake Dr SE Ste 200,nan,nan,Grand Rapids,MI,49546,INS-USG,2020-09-10,49537130317,44903231


C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,ME,addr_city,addr_country,addr_key,addr_line_1,addr_line_2,addr_line_3,addr_plus4,addr_state,addr_type,...,insert_user_id,load_type_ind,me#,processed,source,source_dtm,update_dtm,update_user_id,usage,usage_type
0,02512921232,Scarsdale,NaN,NaN,158 Fox Meadow Rd,nan,nan,NaN,NY,N,...,NaN,R,2512921232,NaN,INS-USG,2020-09-10,NaN,NaN,PP,NaN
1,49514950015,Southlake,NaN,NaN,1804 Leeds Dr,nan,nan,NaN,TX,N,...,NaN,R,49514950015,NaN,INS-USG,2020-09-10,NaN,NaN,PP,NaN
2,02802131341,Ann Arbor,NaN,NaN,1500 E Medical Center Dr,3119 Taubman Center,nan,NaN,MI,N,...,NaN,R,2802131341,NaN,INS-USG,2020-09-10,NaN,NaN,PP,NaN
3,02879111024,Grand Rapids,NaN,NaN,4100 Lake Dr SE Ste 200,nan,nan,NaN,MI,N,...,NaN,R,2879111024,NaN,INS-USG,2020-09-10,NaN,NaN,PP,NaN
4,49537130317,Grand Rapids,NaN,NaN,4100 Lake Dr SE Ste 200,nan,nan,NaN,MI,N,...,NaN,R,49537130317,NaN,INS-USG,2020-09-10,NaN,NaN,PP,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94553,NaN,Greenville,USA,177087,917 W Oak St,nan,nan,nan,MI,N,...,etlaims,R,NaN,Y,INS-USG,9/10/2020 12:00:00 AM,10/20/2020 12:38:03 PM,etlaims,NaN,PP
94554,NaN,Houston,USA,177088,13668 Westheimer Rd,nan,nan,nan,TX,N,...,etlaims,R,NaN,Y,INS-USG,9/10/2020 12:00:00 AM,10/20/2020 12:38:03 PM,etlaims,NaN,PP
94555,NaN,Chicago,USA,177089,1153 N Lavergne Ave,nan,nan,nan,IL,N,...,etlaims,R,NaN,Y,INS-USG,9/10/2020 12:00:00 AM,10/20/2020 12:38:03 PM,etlaims,NaN,PP
94556,NaN,Tarzana,USA,177090,5041 Shirley Ave,nan,nan,nan,CA,N,...,etlaims,R,NaN,Y,INS-USG,9/10/2020 12:00:00 AM,10/20/2020 12:38:03 PM,etlaims,NaN,PP


In [70]:
og_2 = og.fillna('None')

Index(['me#', 'comm_id', 'usage', 'load_type_ind', 'addr_type', 'addr_line_1',
       'addr_line_2', 'addr_line_3', 'addr_city', 'addr_state', 'addr_zip',
       'source', 'source_dtm', 'ME', 'entity_id'],
      dtype='object')

In [78]:
len(og_2[(og_2['addr_line_1']=='nan')&(og_2['addr_line_2']=='nan')&(og_2['addr_line_3']=='nan')])

25

In [79]:
import datacompy
import pandas as pd
compare = datacompy.Compare(
og,
load,
join_columns='entity_id',  #You can also specify a list of columns eg ['policyID','statecode']
abs_tol=0, #Optional, defaults to 0
rel_tol=0, #Optional, defaults to 0
df1_name='Original', #Optional, defaults to 'df1'
df2_name='New' #Optional, defaults to 'df2'
)
print(compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0  Original       15  94877
1       New       22  94558

Column Summary
--------------

Number of columns in common: 12
Number of columns in Original but not in New: 3
Number of columns in New but not in Original: 10

Row Summary
-----------

Matched on: entity_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 94,558
Number of rows in Original but not in New: 319
Number of rows in New but not in Original: 0

Number of rows with some compared columns unequal: 94,558
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 10
Number of columns compared with all values equal: 2
Total number of values which compare unequal: 530,469

Columns with Unequal Values or Types
------------------------------------

        Column Original dtype New dtyp

In [89]:
xx = pd.merge(og, load, on=['entity_id','comm_id'], suffixes = ['_old', '_new'])

In [90]:
x = pd.merge(og, load, on='entity_id', suffixes = ['_old', '_new'])

In [85]:
og.drop_duplicates('comm_id')

,me#,comm_id,usage,load_type_ind,addr_type,addr_line_1,addr_line_2,addr_line_3,addr_city,addr_state,addr_zip,source,source_dtm,me,entity_id
0,2512921232,1448128,PP,R,N,158 Fox Meadow Rd,nan,nan,Scarsdale,NY,10583,INS-USG,2020-09-10,02512921232,5733607
1,49514950015,9461001,PP,R,N,1804 Leeds Dr,nan,nan,Southlake,TX,76092,INS-USG,2020-09-10,49514950015,12753048
2,2802131341,748979,PP,R,N,1500 E Medical Center Dr,3119 Taubman Center,nan,Ann Arbor,MI,48109,INS-USG,2020-09-10,02802131341,43014578
3,2879111024,6107649,PP,R,N,4100 Lake Dr SE Ste 200,nan,nan,Grand Rapids,MI,49546,INS-USG,2020-09-10,02879111024,46605698
5,1606121322,4816062,PP,R,N,5333 McAuley Dr,Rm 3111,nan,Ypsilanti,MI,48197,INS-USG,2020-09-10,01606121322,40432895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94872,3515750036,3141702,PP,R,N,917 W Oak St,nan,nan,Greenville,MI,48838,INS-USG,2020-09-10,03515750036,7488178
94873,49523700471,2951631,PP,R,N,13668 Westheimer Rd,nan,nan,Houston,TX,77077,INS-USG,2020-09-10,49523700471,12827408
94874,30803870582,2862611,PP,R,N,1153 N Lavergne Ave,nan,nan,Chicago,IL,60651,INS-USG,2020-09-10,30803870582,12329105
94875,5104460269,2465546,PP,R,N,5041 Shirley Ave,nan,nan,Tarzana,CA,91356,INS-USG,2020-09-10,05104460269,11136511


In [87]:
xx.columns

Index(['me#', 'comm_id', 'usage', 'load_type_ind_old', 'addr_type_old',
       'addr_line_1_old', 'addr_line_2_old', 'addr_line_3_old',
       'addr_city_old', 'addr_state_old', 'addr_zip_old', 'source_old',
       'source_dtm_old', 'me', 'entity_id_old', 'addr_key', 'addr_line_1_new',
       'addr_line_2_new', 'addr_line_3_new', 'addr_city_new', 'addr_state_new',
       'addr_zip_new', 'addr_plus4', 'addr_country', 'entity_id_new',
       'approval_id', 'processed', 'source_new', 'load_type_ind_new',
       'addr_type_new', 'usage_type', 'source_dtm_new', 'insert_dtm',
       'insert_user_id', 'update_dtm', 'update_user_id'],
      dtype='object')

In [98]:
x[x.entity_id.isin(xx)==False][['addr_line_1_old', 'addr_line_1_new','addr_line_2_new', 'addr_line_2_old','addr_line_3_old','addr_line_3_new','comm_id_old','comm_id_new']]

,addr_line_1_old,addr_line_1_new,addr_line_2_new,addr_line_2_old,addr_line_3_old,addr_line_3_new,comm_id_old,comm_id_new
0,158 Fox Meadow Rd,nan,nan,nan,nan,158 Fox Meadow Rd,1448128,1448128
1,1804 Leeds Dr,nan,nan,nan,nan,1804 Leeds Dr,9461001,9461001
2,1500 E Medical Center Dr,1500 E Medical Center Dr,3119 Taubman Center,3119 Taubman Center,nan,nan,748979,24970372
3,4100 Lake Dr SE Ste 200,nan,nan,nan,nan,4100 Lake Dr SE Ste 200,6107649,6107649
4,4100 Lake Dr SE Ste 200,nan,nan,nan,nan,4100 Lake Dr SE Ste 200,6107649,6107649
...,...,...,...,...,...,...,...,...
94553,917 W Oak St,917 W Oak St,nan,nan,nan,nan,3141702,25062937
94554,13668 Westheimer Rd,13668 Westheimer Rd,nan,nan,nan,nan,2951631,25062938
94555,1153 N Lavergne Ave,1153 N Lavergne Ave,nan,nan,nan,nan,2862611,25062939
94556,5041 Shirley Ave,5041 Shirley Ave,nan,nan,nan,nan,2465546,25062940


In [103]:
xx.columns

Index(['me#', 'comm_id', 'usage', 'load_type_ind_old', 'addr_type_old',
       'addr_line_1_old', 'addr_line_2_old', 'addr_line_3_old',
       'addr_city_old', 'addr_state_old', 'addr_zip_old', 'source_old',
       'source_dtm_old', 'me', 'entity_id', 'addr_key', 'addr_line_1_new',
       'addr_line_2_new', 'addr_line_3_new', 'addr_city_new', 'addr_state_new',
       'addr_zip_new', 'addr_plus4', 'addr_country', 'approval_id',
       'processed', 'source_new', 'load_type_ind_new', 'addr_type_new',
       'usage_type', 'source_dtm_new', 'insert_dtm', 'insert_user_id',
       'update_dtm', 'update_user_id'],
      dtype='object')

In [122]:
comm_id_count = 0
addr_line_1_count = 0
addr_line_2_count = 0
addr_line_3_count = 0
address_count = 0
for row in x.itertuples():
    address = row.addr_line_1_old.upper()
    if row.comm_id_old != row.comm_id_new:
        comm_id_count += 1
    if address not in row.addr_line_1_new.upper():
        addr_line_1_count += 1
    if row.addr_line_2_old.upper() not in row.addr_line_2_new.upper():
        addr_line_2_count += 1
    if row.addr_line_3_old.upper() not in row.addr_line_3_new.upper():
        addr_line_3_count += 1
    if address not in row.addr_line_1_new.upper() and address not in row.addr_line_2_new.upper() and address not in row.addr_line_3_new.upper():
        address_count += 1
        print(address)
        print(row.addr_line_1_new)
        print(row.addr_line_2_new)
        print(row.addr_line_3_new)
        print('')

8875 HIDDEN RIVER PKY
nan
Ste 300                                 
8875 Hidden River Pkwy                  

2500 MORRIS AVE STE 220
Redefine Healthcare                     
Ste 2                                   
2500 Morris Ave                         

9750 NORTH AMERICAN RANCH ROAD
nan
nan
9750 N American Ranch Rd                

APT -H
nan
Apt H                                   
6065 Commanche Ct                       

20318 NORTHPARK DRIVE
nan
nan
20318 Northpark Dr                      

SUITE 100
nan
Ste 100                                 
5901 Vega Ave                           

77 S. DOBSON ROAD
nan
nan
77 S Dobson Rd                          

CHILDREN'S HOSPITAL OF NEW ORLEANS 200 HENRY CLAY AVENUE
Children's Hospital Of New Orleans 200 H
nan
nan

UNIT 618
nan
Apt 618                                 
20 S Stone Ave                          

KHURSHID IQBAL 22841 LARAMIE DR
nan
nan
22841 Laramie Dr                        

5551 HWY. 98
nan
nan
5551 US Highway 98 W     

WALTER REED NATIONAL MILITARY MEDICAL CENTER
Walter Reed National Military Medical Ce
4494 N Palmer Rd                        
nan

WALTER REED NATIONAL MILITARY MEDICAL CENTER
Walter Reed National Military Medical Ce
8901 Rockville Pike                     
nan

WALTER REED NATIONAL MILITARY MEDICAL CENTER
Walter Reed National Military Medical Ce
8901 Rockville Pike                     
nan

103 JOHN WHITAKER BUILDING 500 22ND STREET SOUTH
103 John Whitaker Building 500 22nd Stre
nan
nan

C/O CLEVELAND CLINIC GME DEPT-NA23 9500 EUCLID AVENUE
C/O Cleveland Clinic Gme DEPT-NA23 9500 
nan
nan

C/O CLEVELAND CLINIC GME DEPT-NA23 9500 EUCLID AVENUE
C/O Cleveland Clinic Gme DEPT-NA23 9500 
nan
nan

C/O CLEVELAND CLINIC GME DEPT-NA23 9500 EUCLID AVENUE
C/O Cleveland Clinic Gme DEPT-NA23 9500 
nan
nan

C/O CLEVELAND CLINIC GME DEPT-NA23 9500 EUCLID AVENUE
C/O Cleveland Clinic Gme DEPT-NA23 9500 
nan
nan

C/O CLEVELAND CLINIC GME DEPT-NA23 9500 EUCLID AVENUE
C/O Cleveland Clinic Gme DEPT-NA23 


C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan

C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan

C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan

C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan

C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan

C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan

C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan

C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan

C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan

C/O CCF-GRAD MED EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-Grad Med Edu DEPT-NA23 9500 Eucl
nan
nan


C/O Univ Hosps-Residency Office 11100 Eu
nan
nan

C/O UNIV HOSPS-RESIDENCY OFFICE 11100 EUCLID AVE-RM: LKS 3018
C/O Univ Hosps-Residency Office 11100 Eu
nan
nan

C/O UNIV HOSPS-RESIDENCY OFFICE 11100 EUCLID AVE-RM: LKS 3018
C/O Univ Hosps-Residency Office 11100 Eu
nan
nan

C/O UNIV HOSPS-RESIDENCY OFFICE 11100 EUCLID AVE-RM: LKS 3018
C/O Univ Hosps-Residency Office 11100 Eu
nan
nan

VA MEDICAL CTR MED SERVICE 111(W) 10701 EAST BLVD
Va Medical Ctr Med Service 111(W) 10701 
nan
nan

UNIVERSITY OF CINCINNATI MEDICAL CENTER C/O OBSTETRICS-GYNECOLOGY DEPT 231 ALBER
University Of Cincinnati Medical Center 
nan
nan

MEDICAL COLLEGE OF OHIO GME-ROOM 330-MULFORD LIBRARY 3045 ARLINGTON AVENUE
Medical College Of Ohio Gme-Room 330-Mul
nan
nan

OHIO ST UNIVERSITY HOSP 376 W 105H 016 PRIOR HSL
Ohio St University Hosp 376 W 105H 016 P
nan
nan

UNIV HOSPS/DEPT INTERNAL MEDICINE 231 ALBERT SABIN WAY
Univ Hosps/Dept Internal Medicine 231 Al
nan
nan

C/O CCF-GRAD EDU DEPT-NA23 9500 EUCLID AVENUE
C/O Ccf-

In [147]:
addr_line_1_count

3922

In [134]:
comm_id_count

93261

In [126]:
missing = og[og['me#'].isin(x['me#'])==False]

In [137]:
missing[(missing.addr_state!='~ ')&((missing.addr_line_3!='nan')|(missing.addr_line_1!='nan'))].to_excel('../../Data/AMAIA/Rejects.xlsx', index=False)

In [135]:
len(x)

94558

In [148]:
x[x.entity_id.isin(xx.entity_id)==False][['me','entity_id','comm_id_new','comm_id_old', 
       'addr_line_1_old', 'addr_line_2_old', 'addr_line_3_old',
       'addr_city_old', 'addr_state_old', 'addr_zip_old', 'addr_line_1_new',
       'addr_line_2_new', 'addr_line_3_new', 'addr_city_new', 'addr_state_new',
       'addr_zip_new']].sample(50)

,me,entity_id,comm_id_new,comm_id_old,addr_line_1_old,addr_line_2_old,addr_line_3_old,addr_city_old,addr_state_old,addr_zip_old,addr_line_1_new,addr_line_2_new,addr_line_3_new,addr_city_new,addr_state_new,addr_zip_new
36377,04101010681,8942475,25004761,819149,3400 Spruce St,nan,nan,Philadelphia,PA,19104,3400 Spruce St,nan,nan,Philadelphia,PA,19104
63202,03547130447,43078499,25031586,21535450,901 N Broad St NE Ste 120,Rome Radiology Group Pa,nan,Rome,GA,30161,901 N Broad St NE Ste 120,Rome Radiology Group Pa,nan,Rome,GA,30161
28692,03605980707,8151672,24997076,8954593,Medical Center Blvd,General Internal Medicine,nan,Winston Salem,NC,27157,Medical Center Blvd,General Internal Medicine,nan,Winston Salem,NC,27157
20293,30501130440,43263437,24988677,25062796,421 Langley St,nan,nan,Colorado Spgs,CO,80916,421 Langley St,nan,nan,Colorado Spgs,CO,80916
40219,64914910038,13612491,25008603,896857,836 W Wellington Ave,Ill Masonic Med Cntr,nan,Chicago,IL,60657,836 W Wellington Ave,Ill Masonic Med Cntr,nan,Chicago,IL,60657
42874,02501040293,21751245,25011258,721913,4500 San Pablo Rd S,nan,nan,Jacksonville,FL,32224,4500 San Pablo Rd S,nan,nan,Jacksonville,FL,32224
72332,00506810639,1617027,25040716,24669289,Suite 100,MD Health Clinics 14591 Newport Blvd,nan,Tustin,CA,92780,Suite 100,MD Health Clinics 14591 Newport Blvd,nan,Tustin,CA,92780
90741,00506740606,1605171,25059125,21402590,Univ Of Rochester Med. Ctr.,nan,nan,Rochester,NY,14642,Univ Of Rochester Med. Ctr.,nan,nan,Rochester,NY,14642
26395,03519840510,7645179,24994779,6471649,900 N Washington St,nan,nan,Du Quoin,IL,62832,900 N Washington St,nan,nan,Du Quoin,IL,62832
44924,70402010213,22458783,25013308,7372118,2950 Cleveland Blvd,nan,nan,Weston,FL,33331,2950 Cleveland Blvd,nan,nan,Weston,FL,33331
